In [2]:
import numpy as np
import MDAnalysis as mda
import MDAnalysis.transformations as trans
import nglview as nv
import copy
from ase import io as aio

C:\Users\methy\anaconda3\envs\vasp\Lib\site-packages\Bio\Application\__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [3]:
Path = 'C:/Users/methy/OneDrive - Colostate/GMX/Urea_Sims/MatejCluster/'

u = mda.Universe(Path+'WaterCluster.Anneal.tpr',Path+'WaterCluster.Anneal.trr')


In [4]:
cluster = u.select_atoms('resname ION')
water = u.select_atoms('resname SOL')
whole = u.select_atoms('resname SOL or resname ION')

transforms = [trans.unwrap(cluster),
              trans.center_in_box(cluster,wrap=True),
              trans.wrap(water)]

u.trajectory.add_transformations(*transforms)
    

In [5]:
#view = nv.show_mdanalysis(u)
#view.add_representation('spacefill','resname SOL')
#view.center()
#view

NGLWidget(max_frame=1000)

In [46]:
# writing results to file with on-the-fly transformations, writing to trr
#with mda.Writer(Path+'Matej_Centered.Anneal.trr',whole.n_atoms) as f:
#    for ts in u.trajectory:
#        #u.trajectory[ts.frame]
#        #print(cluster.center_of_mass())
#        f.write(whole)
    

In [47]:
# writing selected frame to PDB file
#with mda.Writer(Path+'GoodStart.pdb',whole.n_atoms) as f:
#    for ts in u.trajectory:
#        if ts.frame == 954:
#            f.write(whole)
#        else:
#            #print(ts.frame)


In [45]:
# writing to POSCAR using ASE utility
# ASE knows how to format it properly and I don't so I can't hardcode it
# HOWEVER, you need to shuffle atoms to appear nicely in POSCAR, so the fanciness here
# is all to shuffle atoms into like kinds as groups

sort = {}
Atomkey = {'POT':'K','OW':'O','HW1':'H','HW2':'H'}

with open(Path+'GoodStart.pdb','r') as fread:
        for line in fread.readlines():
            if 'ATOM' in line.split()[0]:
                if not Atomkey[line.split()[2]] in sort:
                    sort[Atomkey[line.split()[2]]] = [line.rstrip()]
                else:
                    sort[Atomkey[line.split()[2]]].append(line.rstrip())

ordering = ['K','O','H']
# collapsing dictionary into list
atomlist = []
for item in ordering:
    [atomlist.append(sort[item][i]) for i in range(len(sort[item]))]

# renumbering atoms in new order
iterator = 1
for i in range(len(atomlist)):
    line = atomlist[i]
    newline = line.split()[0]+str(iterator).rjust(7)+line[11:]
    atomlist[i] = newline
    iterator += 1

writefile = []
iterator = 0
with open(Path+'GoodStart.pdb','r') as fread:
    with open(Path+'GoodStart_Sorted.pdb','w') as fwrite:
        for line in fread.readlines():
            if 'ATOM' in line.split()[0]:
                writefile.append(atomlist[iterator])
                iterator += 1
            else:
                writefile.append(line.rstrip())
        
        fwrite.writelines(line+'\n' for line in writefile)

Conv = aio.read(Path+'GoodStart_Sorted.pdb',index=0,format='proteindatabank')
Conv.write(Path+'POSCAR.Test',format='vasp')